In [2]:
#importing libraries
from bs4 import BeautifulSoup as bs 
import pandas as pd   
import numpy as np
import os

In [3]:
#Get files names to be able to read each file 
files_names = os.listdir(r'C:\\Users\\hp\\Downloads\\ML_Data\\data')

In [4]:
#Get number of files 
number_of_files = len(files_names) 

In [5]:
#Initiate bs_dict the dictionary that will cotain the data
bs_dict = dict()

In [6]:
#Crate a dictionary with the folder number as the key and the bs object as the value
for file_number in range(number_of_files):
    #get file name in the index of file_number  
    file_name = files_names[file_number]
    #open file according to file_name
    file = open( ('C:\\Users\\hp\\Downloads\\ML_Data\\data\\' + file_name), 'r', encoding = 'utf-8') 
    #set the reference point in the beginning 
    file.seek(0)
    #store file content 
    txt = file.read()
    bs_obj = bs(txt) 
    bs_dict[file_name] = bs_obj

### Delete files dont contain appropriate data("Not founded page in the website")

In [47]:
bs_dict_keys = list(bs_dict.keys())
for file in bs_dict_keys:
    if(bs_dict[file].find(class_ = "notice")):
        bs_dict.pop(file)
    else: 
        continue

In [48]:
#Defining get_value function that will take the feature text, class name of the property and the car id so it can return the value of that feature if exists if not it will return NaN
def get_value(file_number, feature, class_name):
    """
    extract a particular values from a particular file, return nan if it doesnot exist
    inputs:
        file_number(str): file name want to search in it's content
        feature(str): feature name to return it's value
        class_name(str): name of the class the feature exists in
    returns:
        value of the given feature according to file_name, class_name.
        nan if value of the given feature doesnot exist in the file.
    """
    #define additions a list of car additions 
    additions = [ 'مُكيّف', 'إغلاق مركزي', 'جهاز إنذار', 'مسجل CD', 'فتحة سقف', 'جنطات مغنيسيوم', 'فرش جلد', 'وسادة حماية هوائية' ]
    value = np.NaN
    if class_name == "driving-table":
        if feature == 'الموديل':   
            value = bs_dict[file_number].find(class_ = class_name).find("h3").text
        elif feature == 'موديل سنة':
            h5_list = bs_dict[file_number].find(class_ = class_name).findAll("h5")
            for item in h5_list:
                if 'موديل سنة' in item.text:
                    value = int(item.text.split("موديل سنة")[1])
    elif class_name == "post-price":
        value = bs_dict[file_number].find(class_ = class_name).text.split("\n")[0].split(' ')[0]
    else:
        if feature == "سعر التأمين":
            value = int(bs_dict[file_number].findAll(class_ = "list_ads")[1].text.split('\n')[19])
        else:
            features_array = bs_dict[file_number].find(class_ = class_name).text.split("\n")
            if feature in additions:
                if feature in features_array:
                    value = 1 
                else:
                    value = 0
            else:
                for match in features_array: 
                    if feature in match:  
                        value = match.split(feature)[1] 
    return value

In [49]:
#define cars_info a list of dictionaries each one represents a particular car 
cars_info = []
#get features value for each car stored in bs_dict
for current_file_number in list(bs_dict.keys()): 
    cuurent_car_info = {  
        "السعر": float(get_value(current_file_number,"السعر", "post-price")),
        "الموديل": get_value(current_file_number , "الموديل", "driving-table"),
        'موديل سنة': get_value(current_file_number , 'موديل سنة', "driving-table"),
        "لون السيارة": get_value(current_file_number,"لون السيارة", "list_ads"),
        'قوة الماتور': float(get_value(current_file_number,'قوة الماتور', "list_ads")),
        "عدد الركاب": get_value(current_file_number, "عدد الركاب", "list_ads"),  
        "عداد السيارة": get_value(current_file_number,"عداد السيارة", "list_ads"),
        "أصحاب سابقون": get_value(current_file_number, "أصحاب سابقون", "list_ads"),
        'أصل السيارة': get_value(current_file_number,'أصل السيارة', "list_ads"),
        'رخصة السيارة': get_value(current_file_number,'رخصة السيارة', "list_ads"),
        "نوع الوقود": get_value(current_file_number,"نوع الوقود", "list_ads"),
        'نوع الجير': get_value(current_file_number,'نوع الجير', "list_ads"),  
        'الزجاج': get_value(current_file_number,'الزجاج', "list_ads"), 
        "سعر التأمين": get_value(current_file_number, "سعر التأمين", "list_ads"),
        'وسادة حماية هوائية': get_value(current_file_number, 'وسادة حماية هوائية', "list_ads"),
        'فرش جلد': get_value(current_file_number, 'فرش جلد', "list_ads"),   
        'جنطات مغنيسيوم': get_value(current_file_number,'جنطات مغنيسيوم', "list_ads"),
        'فتحة سقف': get_value(current_file_number,'فتحة سقف', "list_ads"),
        'مسجل CD': get_value(current_file_number,'مسجل CD', "list_ads"),    
        'إغلاق مركزي': get_value(current_file_number,'إغلاق مركزي', "list_ads"),
        'مُكيّف': get_value(current_file_number,'مُكيّف', "list_ads"), 
        'جهاز إنذار': get_value(current_file_number,'جهاز إنذار', "list_ads"),
    }   
    cars_info.append(cuurent_car_info)      

In [50]:
#define data, a data frame contains cars data  
data = pd.DataFrame(cars_info)

In [8]:
#Write data to cars_data csv file
data.to_csv('../data/cars_data.csv', index = False)  

,رقم السيارة,السعر,الموديل,موديل سنة,لون السيارة,قوة الماتور,عدد الركاب,الدفع,عداد السيارة,أصحاب سابقون,أصل السيارة,رخصة السيارة,نوع الوقود,نوع الجير,الزجاج,معروضة,وسيلة الدفع
0,500118.txt,100000.0,كيا اوبتيما,2014,أبيض عاجي,2000.0,4+1,نقدا فقط,75000,يد اولى,خصوصي,فلسطينية,بنزين,اوتوماتيك,الكتروني,للبيع فقط,نقدا فقط
1,500163.txt,60000.0,كيا سورينتو,2007,سكني,2500.0,7+1,إمكانية التقسيط,130000,2,خصوصي,فلسطينية,ديزل,نصف اوتوماتيك,الكتروني,للبيع أو التبديل,إمكانية التقسيط
2,500226.txt,43500.0,هونداي افانتي,2006,سكني,1600.0,NaN,نقدا فقط,NaN,NaN,خصوصي,فلسطينية,بنزين,اوتوماتيك,الكتروني,للبيع فقط,نقدا فقط
3,500390.txt,5500.0,فيات 127,1982,بيج,906.0,4+1,إمكانية التقسيط,شغال,00,خصوصي,فلسطينية,بنزين,عادي,يدوي,للبيع فقط,إمكانية التقسيط
4,500393.txt,54000.0,بيجو 208,2014,فضي,1200.0,4+1,نقدا فقط,38000,NaN,خصوصي,فلسطينية,بنزين,اوتوماتيك,الكتروني,للبيع فقط,نقدا فقط
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6859,777734.txt,45000.0,كيا مورننغ,2015,فيراني,1000.0,4+1,إمكانية التقسيط,130000,2,خصوصي,فلسطينية,كهرباء,اوتوماتيك,الكتروني,للبيع فقط,إمكانية التقسيط
6860,777932.txt,48000.0,فورد ترانزيت,2002,أبيض,2400.0,7+1,نقدا فقط,00000,5,عمومي,فلسطينية,ديزل,عادي,الكتروني,للبيع فقط,نقدا فقط
6861,777937.txt,87000.0,بيجو بارتنر,2018,فضي,1600.0,4+1,نقدا فقط,50000,يد صفر,خصوصي,فلسطينية,ديزل,عادي,الكتروني,للبيع فقط,نقدا فقط
6862,777960.txt,126000.0,كيا سورينتو,2017,بترولي,2200.0,6+1,نقدا فقط,100,ثانيه,خصوصي,فلسطينية,ديزل,اوتوماتيك,الكتروني,للبيع فقط,نقدا فقط
